In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, CSVLogger

from loader import load_20news
from custom_layer import MonteCarloLRF, SeparableMonteCarloLRF, SeparableMonteCarloMaxPoolingV2, RandomLRF

import numpy as np
import sklearn as sk
import pickle

/home/vle/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
top_words=10000
sparse=False
remove_short_documents=True
notebook = 'mcNet_top10k_temptative_42'

In [4]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=top_words, sparse=sparse,
                remove_short_documents=remove_short_documents, verbose=False)

In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
num_classes = Y_train.shape[1]

In [7]:
%%time
#Process next cell only once
path = os.path.join('probabilities_' + 
                    'top' + str(top_words) +
                    '_sparse' + str(sparse) +
                    '_removeShorts' + str(remove_short_documents) +
                    '.pkl')
if os.path.isfile(path):
  probabilities = pickle.load(open(path, "rb"), encoding='latin1')

CPU times: user 2.3 s, sys: 728 ms, total: 3.03 s
Wall time: 3.03 s


In [8]:
if not(os.path.isfile(path)):
  METRIC = 'cosine'#'euclidean'
  distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

  # enforce exact zero
  for k in xrange(distances.shape[0]):
    distances[k,k] = 0.

  # max normalize
  #distances /= distances.max()
  distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

  # use tricube kernel (becaause of flatness around 0)
  probabilities = (1. - np.abs(distances) ** 3) ** 3

  # remove auto connections (which are taken anyway in LRF)
  for k in xrange(probabilities.shape[0]):
    probabilities[k,k] = 0.

  # normalize proba
  probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))
  
  # pickled for later use
  pickle.dump(probabilities, open(path,"wb"))  

In [9]:
batch_size = 128
model = Sequential()
model.add(RandomLRF(probabilities, LRF_size=16, filters=25, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(Conv1D(1, kernel_size=1, activation='relu',
#                 padding='same',kernel_initializer='he_uniform'))
#model.add(SeparableMonteCarloMaxPoolingV2(LRF_size=4, new_size=2500))
#model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Dense(500, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_lrf_1 (RandomLRF)     (None, 10000, 25)         425       
_________________________________________________________________
flatten_1 (Flatten)          (None, 250000)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 250000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                5000020   
Total params: 5,000,445
Trainable params: 5,000,445
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopper = EarlyStopping(min_delta=0.001, patience=2)
csv = CSVLogger(notebook + '_log.csv')

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    #callbacks=[early_stopper, csv],
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10168 samples, validate on 7071 samples
Epoch 1/20
10168/10168 [==============================] - 6s 555us/step - loss: 2.8423 - acc: 0.2796 - val_loss: 2.6744 - val_acc: 0.4497
Epoch 2/20
10168/10168 [==============================] - 4s 417us/step - loss: 2.3794 - acc: 0.6125 - val_loss: 2.2946 - val_acc: 0.4900
Epoch 3/20
10168/10168 [==============================] - 4s 417us/step - loss: 1.8677 - acc: 0.7413 - val_loss: 1.9204 - val_acc: 0.6092
Epoch 4/20
10168/10168 [==============================] - 4s 418us/step - loss: 1.4097 - acc: 0.8099 - val_loss: 1.6228 - val_acc: 0.6580
Epoch 5/20
10168/10168 [==============================] - 4s 417us/step - loss: 1.0705 - acc: 0.8538 - val_loss: 1.4317 - val_acc: 0.6798
Epoch 6/20
10168/10168 [==============================] - 4s 417us/step - loss: 0.8284 - acc: 0.8816 - val_loss: 1.3152 - val_acc: 0.6877
Epoch 7/20
10168/10168 [==============================] - 4s 418us/step - loss: 0.6667 - acc: 0.9025 - val_loss: 1.2418 - v

In [11]:
%%bash
tail mcNet_top10k_temptative_42_log.csv

5,0.8815892997170696,0.828352999537128,0.6877386508694702,1.3152040708454473
6,0.9025373721010191,0.6667481069486034,0.6884457642970371,1.2418389436699953
7,0.9173878835562549,0.5414071554841252,0.6996181587100012,1.1848604357271244
8,0.9331235246898432,0.4520701199043088,0.7028708809623454,1.1558573319338417
9,0.943646734995133,0.37929777361388095,0.6997595814224888,1.1430704626701893
10,0.9557435090948896,0.31838071144630736,0.7014566538458986,1.1294353204507872
11,0.9633162863886703,0.27186571333607196,0.7032951492346795,1.1242104254725158
12,0.9709874115808098,0.23394882282715534,0.704143685374734,1.1229980590741009
13,0.9754130606291146,0.20216404267364746,0.70386084008463,1.1292391612107946
14,0.9818056649246336,0.173333864644416,0.7013152311249815,1.1366051992151496
